In [ ]:
import pywatershed
import pandas as pd
import pathlib as pl
import json
import numpy as np
from pywatershed.parameters.prms_parameters import JSONParameterEncoder

import sys
sys.path.append('../scripts/')
from pest_utils import pars_to_tpl_entries
sys.path.append('../dependencies/')
import pyemu

interrupt_notebook = False

In [ ]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [ ]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [ ]:
cm = all_models[2] # sets cutout from list
pestdir = rootdir/ cm # stes path to location of NHM output folder where output files are.

In [ ]:
pst = pyemu.Pst(str(pestdir / 'prior_mc.pst'))

In [ ]:
pst.plot(kind='phi_pie')

In [ ]:
pst.phi_components

In [ ]:
phi = pst.phi
phi

In [ ]:
#stop

In [ ]:
phi_new_comps = {'actet_mean_mon':0.05 * phi,
 'actet_mon':  .10 * phi,
 'streamflow_nodata': 0 * phi ,
 'recharge_ann': 0.05 * phi,
 'runoff_mon': .15 * phi ,
 'sca_daily':0 * phi ,
 'soil_moist_ann': 0.05 * phi,
 'soil_moist_mon': 0.05 * phi ,
 'streamflow_mean_mon': .10 * phi ,
 'streamflow_mon': .10 * phi }

In [ ]:
# add streamflow_daily components
stream_comps_perc = 0.35
stream_comps = len([i for i,_ in pst.phi_components.items() if i.lower().startswith('streamflow_daily')])
stream_comps_each_perc = stream_comps_perc / stream_comps

for i in pst.phi_components.keys():
    if i.lower().startswith('streamflow_daily'):
        phi_new_comps[i] = stream_comps_each_perc * phi

In [ ]:
try:
    assert np.isclose(sum([i for _,i in phi_new_comps.items()])/phi, 1.0)
    print('Very very close to 1.0. Carry on!')
except:
    print('Not close to 1.0. Stop and recheck your weight for catagory.')

In [ ]:
sum([i for _,i in phi_new_comps.items()])/phi

In [ ]:
assert np.isclose(sum([i for _,i in phi_new_comps.items()])/phi, 1.0)

In [ ]:
pst.obs_groups

In [ ]:
pst.adjust_weights(obsgrp_dict=phi_new_comps)

In [ ]:
pst.plot(kind='phi_pie')

In [ ]:
pst.observation_data.loc[pst.observation_data.obgnme=='streamflow_nodata']

In [ ]:
#Write a new version of the PEST++ control file (.pst)
pst.write(str(rootdir/cm/ 'prior_mc_reweight.pst'), version=2)


In [ ]:
print(pst.adjust_weights(obsgrp_dict=phi_new_comps))


In [ ]:
#Write this to a dataframe (.csv) and save it somewhere so that we have a record of it. with time date and append to .csv every notebook run.
print(pst.phi_components_normalized)
